In [2]:
import pandas as pd

countries = pd.read_csv("Countries1.csv")
country_codes = countries["Alpha2"].tolist() # Get country code column

In [3]:
import requests
from pathlib import Path

session = requests.Session()

base_url = "https://data.countrydata.iatistandard.org/output/web/xlsx/en"
out_dir = Path("data/country_xlsx")
out_dir.mkdir(parents=True, exist_ok=True)

failed = []

for code in country_codes:
    url = f"{base_url}/{code}.xlsx"
    out_path = out_dir / f"{code}.xlsx"

    if out_path.exists():
        continue

    try:
        r = session.get(url, timeout=60)
        r.raise_for_status()

        with open(out_path, "wb") as f:
            f.write(r.content)

    except requests.exceptions.HTTPError:
        failed.append(code)

print("Done.")
print("Failed codes:", failed)


Done.
Failed codes: ['JG', nan]


In [4]:
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
INPUT_DIR = Path("data/country_xlsx")
OUTPUT_DIR = Path("data/cleaned_country_xlsx")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Columns to remove
COLUMNS_TO_REMOVE = ['Humanitarian', 'Multi Country', 'URL', 'Value (EUR)', 'Value (Local currrency)']

# Expected date format for database
DATE_FORMAT = '%Y-%m-%d'


def clean_text_field(text): # Standardize text fields: trim whitespace, normalize encoding

    if pd.isna(text):
        return text

    text = str(text).strip()

    # Handle 'No data' or similar placeholders
    if text.lower() in ['no data', 'n/a', 'na', 'null', '']:
        return None

    return text


def normalize_country_name(country):
    # Normalize country spellings and names
    # Common corrections for misspellings

    if pd.isna(country):
        return country

    country = str(country).strip()

    # Dictionary of common misspellings/variations
    corrections = {
        'Turkiye': 'Turkey',
        'Türkiye': 'Turkey',
        'Ivory Coast': "Côte d'Ivoire",
        'Congo DRC': 'Democratic Republic of the Congo',
        'Congo Republic': 'Republic of the Congo',
    }

    for wrong, correct in corrections.items():
        if wrong.lower() in country.lower():
            return correct

    return country


def validate_numeric_values(df):
    # Validate that numeric fields have acceptable values
    # Remove/correct impossible values (e.g., negative values where positive expected)

    # Value columns should not be negative (assuming all transactions are positive)
    value_columns = ['Value (USD)']

    for col in value_columns:
        if col in df.columns:
            # Flag negative values
            negative_mask = df[col] < 0
            if negative_mask.any():
                print(f"  WARNING: Found {negative_mask.sum()} negative values in {col}")
                # Option 1: Set to absolute value
                # df.loc[negative_mask, col] = df.loc[negative_mask, col].abs()
                # Option 2: Set to NaN
                df.loc[negative_mask, col] = np.nan

    return df


def remove_duplicates(df): #Remove duplicate rows based on key columns

    # Define key columns that should be unique
    # Rows that are duplicated in all these columns will be considered duplicates
    key_columns = [
        'IATI Identifier',
        'Transaction Type',
        'Calendar Year',
        'Calendar Quarter',
        'Value (USD)'
    ]

    # Check columns exist
    existing_keys = [col for col in key_columns if col in df.columns]

    initial_count = len(df)
    df = df.drop_duplicates(subset=existing_keys, keep='first')
    removed = initial_count - len(df)

    if removed > 0:
        print(f"  Removed {removed} duplicate rows")

    return df


def clean_and_transform_data(file_path):
    # Main cleaning and transformation function for a single country file

    country_code = file_path.stem  # e.g., 'AF' from 'AF.xlsx'
    print(f"\nProcessing: {country_code}.xlsx")

    try:
        # Read the Excel file
        df = pd.read_excel(file_path)
        print(f"  Initial rows: {len(df)}")

        # 1. Remove specified columns
        columns_to_drop = [col for col in COLUMNS_TO_REMOVE if col in df.columns]
        if columns_to_drop:
            df = df.drop(columns=columns_to_drop)
            print(f"  Removed columns: {', '.join(columns_to_drop)}")

        # 2. Clean text fields
        text_columns = df.select_dtypes(include=['object']).columns
        for col in text_columns:
            df[col] = df[col].apply(clean_text_field)

        # 3. Normalize country names
        if 'Recipient Country or Region' in df.columns:
            df['Recipient Country or Region'] = df['Recipient Country or Region'].apply(
                normalize_country_name
            )

        # 4. Validate numeric values
        df = validate_numeric_values(df)

        # 5. Remove duplicates
        df = remove_duplicates(df)

        # 6. Handle missing values in key fields
        # Option: Drop rows with missing critical fields
        critical_fields = ['IATI Identifier', 'Transaction Type', 'Calendar Year']
        before_drop = len(df)
        df = df.dropna(subset=critical_fields)
        dropped = before_drop - len(df)
        if dropped > 0:
            print(f"Dropped {dropped} rows with missing critical fields")

        # 7. Sort by date and identifier
        sort_columns = ['Transaction Date', 'IATI Identifier']
        existing_sort = [col for col in sort_columns if col in df.columns]
        if existing_sort:
            df = df.sort_values(by=existing_sort)

        # 8. Reset index
        df = df.reset_index(drop=True)

        # Save cleaned data
        output_file = OUTPUT_DIR / f"{country_code}_cleaned.csv"
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"  Final rows: {len(df)}")
        print(f"  Saved to: {output_file.name}")

        return {
            'country': country_code,
            'initial_rows': len(df),
            'final_rows': len(df),
            'success': True
        }

    except Exception as e:
        print(f"  ERROR: {str(e)}")
        return {
            'country': country_code,
            'success': False,
            'error': str(e)
        }

def main():
    # Main execution function

    print("="*60)
    print("IATI DATA CLEANING AND TRANSFORMATION")
    print("="*60)
    print(f"Input directory: {INPUT_DIR}")
    print(f"Output directory: {OUTPUT_DIR}")

    # Get all Excel files (country files)
    excel_files = list(INPUT_DIR.glob("*.xlsx"))
    
    file_path = INPUT_DIR / "AF.xlsx"

    if not file_path.exists():
        print(f"\nERROR: File not found: {file_path}")
        return

    clean_and_transform_data(file_path)

    print("\n" + "="*60)
    print("PROCESSING COMPLETE!")
    print("="*60)


if __name__ == "__main__":
    main()

IATI DATA CLEANING AND TRANSFORMATION
Input directory: data/country_xlsx
Output directory: data/cleaned_country_xlsx

Processing: AF.xlsx
  Initial rows: 133277
  Removed columns: Humanitarian, Multi Country, URL, Value (EUR), Value (Local currrency)
  Removed 9345 duplicate rows
  Final rows: 123932
  Saved to: AF_cleaned.csv

PROCESSING COMPLETE!


In [7]:
import pandas as pd

# Path to cleaned file
clean_file = OUTPUT_DIR / "AF_cleaned.csv"   # adjust name if needed

# Load cleaned data
df_clean = pd.read_csv(clean_file)

# Ensure value column is numeric
df_clean['Value (USD)'] = pd.to_numeric(df_clean['Value (USD)'], errors='coerce')

# Check for negative values
negative_rows = df_clean[df_clean['Value (USD)'] < 0]

print(f"Negative values found: {len(negative_rows)}")

if not negative_rows.empty:
    print("\nSample negative-value rows:")
    print(
        negative_rows[
            ['IATI Identifier', 'Transaction Type', 'Transaction Date']
        ].head()
    )
else:
    print("No negative values found 🎉")


Negative values found: 0
No negative values found 🎉
